## prune to vpu09

In [4]:
import os,sys,git,tempfile,psycopg2;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
from contextlib import closing;

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pasw = os.environ['POSTGRESQL_CIP_PASS'];

In [5]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pasw
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

database is ready


In [ ]:
with closing(conn.cursor()) as cursor:
    
    ###############################################################################
    # MR
    # Remove all but VPU 09
    cursor.execute("DELETE FROM cipsrv_epageofab_m.catchment_fabric a WHERE SUBSTR(a.vpuid,1,2) != '09'");
    conn.commit();
    
    # Remove all xwalk but for catchments in VPU09
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_m.catchment_fabric_xwalk a WHERE NOT EXISTS(
            SELECT 1 FROM cipsrv_epageofab_m.catchment_fabric b WHERE b.nhdplusid = a.nhdplusid
        )    
    """);
    conn.commit(); 
    
    # Move all precalculated xwalk polygons but for xwalk catchments in VPU09
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_m.catchment_fabric_huc12_np21 a WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_epageofab_m.catchment_fabric_xwalk b WHERE b.xwalk_huc12_version = 'NP21' AND b.xwalk_huc12 = a.xwalk_huc12
        )
    """);
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_m.catchment_fabric_huc12_nphr a WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_epageofab_m.catchment_fabric_xwalk b WHERE b.xwalk_huc12_version = 'NPHR' AND b.xwalk_huc12 = a.xwalk_huc12
        )
    """);
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_m.catchment_fabric_huc12_f3   a WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_epageofab_m.catchment_fabric_xwalk b WHERE b.xwalk_huc12_version = 'F3' AND b.xwalk_huc12 = a.xwalk_huc12
        )
    """);
    conn.commit();

    cursor.execute("TRUNCATE TABLE cipsrv_epageofab_m.alaskacatchment");
    cursor.execute("TRUNCATE TABLE cipsrv_epageofab_m.oceancatchment");    
    
    # Prune other NHDPlus table to MR VPU09
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.networknhdflowline    a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.nonnetworknhdflowline a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_m.nhdplusflow           a WHERE SUBSTR(a.fromvpuid,1,2) != '09'");
    conn.commit();
    
    ###############################################################################
    # HR
    # Remove all but VPU 09
    cursor.execute("DELETE FROM cipsrv_epageofab_h.catchment_fabric a WHERE SUBSTR(a.vpuid,1,2) != '09'");
    conn.commit();

    # Remove all xwalk but for catchments in VPU09
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_h.catchment_fabric_xwalk a WHERE NOT EXISTS(
            SELECT 1 FROM cipsrv_epageofab_h.catchment_fabric b WHERE b.nhdplusid = a.nhdplusid
        )    
    """);
    conn.commit();   

    # Move all precalculated xwalk polygons but for xwalk catchments in VPU09
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_h.catchment_fabric_huc12_np21 a WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_epageofab_h.catchment_fabric_xwalk b WHERE b.xwalk_huc12_version = 'NP21' AND b.xwalk_huc12 = a.xwalk_huc12
        )
    """);
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_h.catchment_fabric_huc12_nphr a WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_epageofab_h.catchment_fabric_xwalk b WHERE b.xwalk_huc12_version = 'NPHR' AND b.xwalk_huc12 = a.xwalk_huc12
        )
    """);
    cursor.execute("""
        DELETE FROM cipsrv_epageofab_h.catchment_fabric_huc12_f3   a WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_epageofab_h.catchment_fabric_xwalk b WHERE b.xwalk_huc12_version = 'F3' AND b.xwalk_huc12 = a.xwalk_huc12
        )
    """);
    conn.commit();    

    cursor.execute("TRUNCATE TABLE cipsrv_epageofab_h.alaskacatchment");
    cursor.execute("TRUNCATE TABLE cipsrv_epageofab_h.grid0catchment");
    cursor.execute("TRUNCATE TABLE cipsrv_epageofab_h.oceancatchment");  
    
    # Prune other NHDPlus table to HR VPU09
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.networknhdflowline    a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.nonnetworknhdflowline a WHERE SUBSTR(a.vpuid,1,2)     != '09'");
    cursor.execute("DELETE FROM cipsrv_nhdplus_h.nhdplusflow           a WHERE SUBSTR(a.fromvpuid,1,2) != '09'");
    conn.commit();

    ###############################################################################
    # Support
    cursor.execute("""
       DELETE FROM cipsrv_support.tiger_aiannha_5070 a 
       WHERE NOT ST_INTERSECTS(
           a.shape
          ,(SELECT ST_COLLECT(b.shape) FROM cipsrv_support.tiger_fedstatewaters_5070 b WHERE b.stusps IN ('MN','ND','SD'))
       )""");
    conn.commit();
    
    cursor.execute("""
        DELETE FROM cipsrv_support.tiger_aiannha a 
        WHERE NOT EXISTS (
            SELECT 1 FROM cipsrv_support.tiger_aiannha_5070 b WHERE b.geoid = a.geoid
        )""");
    
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_3338");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_26904");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_32161");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_32655");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_aiannha_32702");
    conn.commit();
    
    cursor.execute("DELETE FROM cipsrv_support.tiger_fedstatewaters a WHERE a.stusps NOT IN ('MN','ND','SD')");
    cursor.execute("DELETE FROM cipsrv_support.tiger_fedstatewaters_5070 a WHERE a.stusps NOT IN ('MN','ND','SD')");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_3338");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_26904");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_32161");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_32655");
    cursor.execute("TRUNCATE TABLE cipsrv_support.tiger_fedstatewaters_32702");
    conn.commit();
    
    ###############################################################################
    # WBD
    cursor.execute("DELETE FROM cipsrv_wbd.wbd_hu12_f3  a WHERE SUBSTR(a.huc12,1,2) != '09'");
    conn.commit();

    cursor.execute("DELETE FROM cipsrv_wbd.wbd_hu12_np21 a WHERE SUBSTR(a.huc12,1,2) != '09'");
    conn.commit();

    cursor.execute("DELETE FROM cipsrv_wbd.wbd_hu12_nphr a WHERE SUBSTR(a.huc12,1,2) != '09'");
    conn.commit();


In [ ]:
conn.close();
